In [1]:
!pip install pyautogen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.3/497.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.2/314.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.6 MB/s eta 0:00:00


In [2]:
#importing the libraries
import copy
import json
import os
from typing import Any , Callable , Dict , List , Literal , Optional , Tuple , Union

from openai import BadRequestError

import autogen
from autogen import config_list_from_json
from autogen.agentchat import Agent
from autogen.agentchat.contrib.agent_optimizer import AgentOptimizer
from autogen.agentchat.contrib.math_user_proxy_agent import MathUserProxyAgent
from autogen.code_utils import extract_code
from autogen.math_utils import get_answer

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
#Using the Math User Proxy with function_call
def is_termination_msg_mathchat(message): #this method is called for the purpose of checking th termination method
    """Check if a message is a termination message."""
    if isinstance(message, dict): #If the message if of dictionary type then only it will check
        message = message.get("content") #taking the content key from dictionary
        if message is None: #If None msg is there then it will return false
            return False
    cb = extract_code(message) #Extracting the python code snippet from the message
    contain_code = False #purpose of this variable to tracj whether any python code snipet is present in the message or not . oif its present then ot it will automatically change to True
    for c in cb: #using the loop for the extracted code snippets
        if c[0] == "python": #if the snippet code is python then it will check the condition
            contain_code = True #after getting the python code it will set contain_code as true
            break
    if message.rstrip().find("TERMINATE") >= 0:
        return True #if message is TERMINATED it will retrun True
    return not contain_code and get_answer(message) is not None and get_answer(message) != "" #if there is no code and valid answer then it will return true


#using MathUserProxyAgent for solving math related problem
class MathUserProxyAgent(MathUserProxyAgent):
    MAX_CONSECUTIVE_AUTO_REPLY = 15
    DEFAULT_REPLY = "Continue. Please keep solving the problem until you need to query. (If you get to the answer, put it in \\boxed{}.)"
    PROMPTS = """Let's solve a math problem.
Query requirements:
You should always use the 'print' function for the output and use fractions/radical forms instead of decimals.
You can use packages like sympy to help you.
You must follow the formats below to write your code:
```python
# your code
```
If some packages are missing, you could also suggest a code to install the corresponding package.

Please follow this process:
1. Solve the problem step by step (do not over-divide the steps).
2. Take out any queries that can be asked through Python code (for example, any calculations or equations that can be calculated) and functions you know in the context of this conversation.

Please
(1) do not mix suggested Python codes and function calls in one step.
(2) You MUST remember that you don’t have a function named "python" available.

You must follow the formats below to write your Python code:
```python
# your code
```

3. Wait for me to give the results or wait for the executed results of the function call.
4. Continue if you think the result is correct. If the result is invalid or unexpected, please correct your query or reasoning.

After all the queries are run and you get the answer, put the answer in \\boxed{}.

Problem:
"""

    def __init__(
        self,
        name: Optional[str] = "MathChatAgent",
        is_termination_msg: Optional[Callable[[Dict], bool]] = is_termination_msg_mathchat,
        human_input_mode: Literal["ALWAYS", "NEVER", "TERMINATE"] = "NEVER",
        default_auto_reply: Optional[Union[str, Dict, None]] = DEFAULT_REPLY,
        max_invalid_q_per_step=3,
        **kwargs,
    ):
        super().__init__(
            name=name,
            is_termination_msg=is_termination_msg,
            human_input_mode=human_input_mode,
            default_auto_reply=default_auto_reply,
            max_invalid_q_per_step=max_invalid_q_per_step,
            **kwargs,
        )
        del self._reply_func_list[2]
        self.register_reply([Agent, None], MathUserProxyAgent._generate_math_reply, position=4)
        del self._reply_func_list[3]
        self.register_reply(
            trigger=autogen.ConversableAgent, reply_func=MathUserProxyAgent.generate_function_call_reply, position=3
        )
        self.register_reply(
            trigger=autogen.ConversableAgent, reply_func=MathUserProxyAgent._check_final_result, position=0
        )

        self.max_function_call_trial = 3
        self.query = None
        self.answer = None
        self.is_correct = None

#generating function resposes , execute function  , and thells about the success and failure of the function
    def generate_function_call_reply(
        self,
        messages: Optional[List[Dict]] = None,
        sender: Optional[autogen.ConversableAgent] = None,
        config: Optional[Any] = None,
    ) -> Tuple[bool, Union[Dict, None]]:
        """Generate a reply using function call."""
        if messages is None:
            messages = self._oai_messages[sender]
        message = messages[-1]
        if "function_call" in message:
            is_exec_success, func_return = self.execute_function(message["function_call"])
            if is_exec_success:
                self.max_function_call_trial = 3
                return True, func_return
            else:
                if self.max_function_call_trial == 0:
                    error_message = func_return["content"]
                    self.max_function_call_trial = 3
                    return (
                        True,
                        "The func is executed failed many times. "
                        + error_message
                        + ". Please directly reply me with TERMINATE. We need to terminate the conversation.",
                    )
                else:
                    revise_prompt = "You may make a wrong function call (It may due the arguments you provided doesn't fit the function arguments like missing required positional argument). \
                    If you think this error occurs due to you make a wrong function arguments input and you could make it success, please try to call this function again using the correct arguments. \
                    Otherwise, the error may be caused by the function itself. Please directly reply me with TERMINATE. We need to terminate the conversation. "
                    error_message = func_return["content"]
                    return True, "The func is executed failed." + error_message + revise_prompt
        return False, None

    def initiate_chat(
        self,
        recipient,
        answer: None,
        silent: Optional[bool] = False,
        **context,
    ):
        self.query = context["problem"]
        if not isinstance(answer, str):
            answer = str(answer)
            if answer.endswith(".0"):
                answer = answer[:-2]
            self._answer = answer
        else:
            self._answer = answer

        self.is_correct = None

        self._prepare_chat(recipient, True)
        error_message = None
        try:
            prompt = self.PROMPTS + context["problem"]
            self.send(prompt, recipient, silent=silent)
        except BadRequestError as e:
            error_message = str(e)
            self.is_correct = 0
            print("error information: {}".format(error_message))

        recipient.reset()
        is_correct = copy.deepcopy(self.is_correct)
        self._reset()
        return is_correct

    def _check_final_result(
        self,
        messages: Optional[List[Dict]] = None,
        sender: Optional[autogen.Agent] = None,
        config: Optional[Any] = None,
    ):
        messages = messages[-1]
        if isinstance(messages, dict):
            messages = messages.get("content")
            if messages is None:
                return False, None

        cb = extract_code(messages)
        contain_code = False
        for c in cb:
            if c[0] == "python":
                contain_code = True
                break
        if not contain_code and get_answer(messages) is not None and get_answer(messages) != "":
            if get_answer(messages) == self._answer:
                self.is_correct = 1
                return True, "The result is Correct. Please reply me with TERMINATE."
            else:
                self.is_correct = 0
                return False, None
        else:
            return False, None

    def _reset(self):
        super()._reset()
        self.max_function_call_trial = 3
        self.is_correct = None
        self.query = None
        self.answer = None

In [5]:
#Loading the dataset
test_data, train_data = [], []
with open("algebra.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        test_data.append(json.loads(line))
with open("2algebra.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        train_data.append(json.loads(line))
test_data, train_data = test_data[0:10], train_data[0:10]

In [13]:
#Agent Construction
llm_config = {
    "config_list": [
        {
            "model":"gpt-4o-mini",
            "api_type":"open_ai",
            "api_key": "...",
        }
    ]
}

assistant = autogen.AssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant.",
    llm_config=llm_config,
)
user_proxy = MathUserProxyAgent(
    name="mathproxyagent",
    human_input_mode="NEVER",
    code_execution_config={"work_dir": "_output", "use_docker": False},
)


In [14]:
#testing with optimization first
sum = 0
for index, query in enumerate(test_data): #It loops through each query in test_data using enumerate.
    is_correct = user_proxy.initiate_chat(recipient=assistant, answer=query["answer"], problem=query["question"]) #
    print(is_correct)
    sum += is_correct #The sum variable accumulates the count of correct answers.
success_rate_without_agent_training = sum / 10

mathproxyagent (to assistant):

Let's solve a math problem.
Query requirements:
You should always use the 'print' function for the output and use fractions/radical forms instead of decimals.
You can use packages like sympy to help you.
You must follow the formats below to write your code:
```python
# your code
```
If some packages are missing, you could also suggest a code to install the corresponding package.

Please follow this process:
1. Solve the problem step by step (do not over-divide the steps).
2. Take out any queries that can be asked through Python code (for example, any calculations or equations that can be calculated) and functions you know in the context of this conversation.

Please
(1) do not mix suggested Python codes and function calls in one step.
(2) You MUST remember that you don’t have a function named "python" available.

You must follow the formats below to write your Python code:
```python
# your code
```

3. Wait for me to give the results or wait for the exec

In [23]:
# Agent Training
EPOCH = 5
optimizer_model = "gpt-4o-mini"
optimizer = AgentOptimizer(max_actions_per_step=3, llm_config=llm_config, optimizer_model=optimizer_model)
for i in range(EPOCH):
    for index, query in enumerate(train_data):
        is_correct = user_proxy.initiate_chat(assistant, answer=query["answer"], problem=query["question"])
        history = assistant.chat_messages_for_summary(user_proxy)
        optimizer.record_one_conversation(history, is_satisfied=is_correct)
    register_for_llm, register_for_exector = optimizer.step()
    for item in register_for_llm:
        assistant.update_function_signature(**item)
    if len(register_for_exector.keys()) > 0:
        user_proxy.register_function(function_map=register_for_exector)

mathproxyagent (to assistant):

Let's solve a math problem.
Query requirements:
You should always use the 'print' function for the output and use fractions/radical forms instead of decimals.
You can use packages like sympy to help you.
You must follow the formats below to write your code:
```python
# your code
```
If some packages are missing, you could also suggest a code to install the corresponding package.

Please follow this process:
1. Solve the problem step by step (do not over-divide the steps).
2. Take out any queries that can be asked through Python code (for example, any calculations or equations that can be calculated) and functions you know in the context of this conversation.

Please
(1) do not mix suggested Python codes and function calls in one step.
(2) You MUST remember that you don’t have a function named "python" available.

You must follow the formats below to write your Python code:
```python
# your code
```

3. Wait for me to give the results or wait for the exec

/usr/local/lib/python3.10/dist-packages/autogen/agentchat/conversable_agent.py:2553: UserWarning: Function 'general_quadratic_solver' is being overridden.
  warnings.warn(f"Function '{func_sig['name']}' is being overridden.", UserWarning)


mathproxyagent (to assistant):

Let's solve a math problem.
Query requirements:
You should always use the 'print' function for the output and use fractions/radical forms instead of decimals.
You can use packages like sympy to help you.
You must follow the formats below to write your code:
```python
# your code
```
If some packages are missing, you could also suggest a code to install the corresponding package.

Please follow this process:
1. Solve the problem step by step (do not over-divide the steps).
2. Take out any queries that can be asked through Python code (for example, any calculations or equations that can be calculated) and functions you know in the context of this conversation.

Please
(1) do not mix suggested Python codes and function calls in one step.
(2) You MUST remember that you don’t have a function named "python" available.

You must follow the formats below to write your Python code:
```python
# your code
```

3. Wait for me to give the results or wait for the exec

/usr/local/lib/python3.10/dist-packages/autogen/agentchat/conversable_agent.py:2514: UserWarning: Function 'general_quadratic_solver' is being overridden.
  warnings.warn(f"Function '{name}' is being overridden.", UserWarning)


assistant (to mathproxyagent):

To ensure that the piecewise function \( f(x) \) is continuous, we need to check the continuity at the points where the function definition changes, specifically at \( x = 2 \) and \( x = -2 \).

### Step 1: Set up the continuity equations

1. **At \( x = 2 \)**:

   For continuity at \( x = 2 \):
   \[
   ax + 3 = x - 5
   \]
   Substituting \( x = 2 \):
   \[
   2a + 3 = 2 - 5
   \]

2. **At \( x = -2 \)**:

   For continuity at \( x = -2 \):
   \[
   x - 5 = 2x - b
   \]
   Substituting \( x = -2 \):
   \[
   -2 - 5 = -4 - b
   \]

### Step 2: Solve the equations

Now we can express these equations in Python and solve for \( a \) and \( b \).

```python
from sympy import symbols, Eq, solve

# Define variables
a, b = symbols('a b')

# Equation at x = 2
eq1 = Eq(2*a + 3, -3)  # because 2 - 5 = -3

# Equation at x = -2
eq2 = Eq(-2 - 5, -4 - b)  # because -4 can be rearranged to solve for b

# Solve the equations
solution = solve((eq1, eq2), (a, b))
solut

/usr/local/lib/python3.10/dist-packages/autogen/agentchat/conversable_agent.py:2553: UserWarning: Function 'evaluate_arithmetic_sequence' is being overridden.
  warnings.warn(f"Function '{func_sig['name']}' is being overridden.", UserWarning)
/usr/local/lib/python3.10/dist-packages/autogen/agentchat/conversable_agent.py:2514: UserWarning: Function 'evaluate_arithmetic_sequence' is being overridden.
  warnings.warn(f"Function '{name}' is being overridden.", UserWarning)


assistant (to mathproxyagent):

To solve the problem of finding \(a + b\) such that the piecewise function 

\[
f(x) = \left\{
\begin{array}{cl} ax+3, &\text{ if }x>2, \\
x-5 &\text{ if } -2 \le x \le 2, \\
2x-b &\text{ if } x <-2.
\end{array}
\right.
\]

is continuous, we need to ensure that the function values match at the points where the pieces of the function change, which are at \(x = -2\) and \(x = 2\).

### Step 1: Continuity at \(x = 2\)

First, we will check the continuity at \(x = 2\):

1. From \(x > 2\), we have \(f(2) = a(2) + 3 = 2a + 3\).
2. From \(-2 \leq x \leq 2\), we have \(f(2) = 2 - 5 = -3\).

For continuity at \(x = 2\), we set these two expressions equal:

\[
2a + 3 = -3.
\]

### Step 2: Solve for \(a\)

Let's solve for \(a\):

\[
2a + 3 = -3 \\
2a = -3 - 3 \\
2a = -6 \\
a = -3.
\]

### Step 3: Continuity at \(x = -2\)

Next, we will check the continuity at \(x = -2\):

1. From \(-2 \leq x \leq 2\), we have \(f(-2) = -2 - 5 = -7\).
2. From \(x < -2\), we have \(f

/usr/local/lib/python3.10/dist-packages/autogen/agentchat/conversable_agent.py:2553: UserWarning: Function 'evaluate_arithmetic_terms' is being overridden.
  warnings.warn(f"Function '{func_sig['name']}' is being overridden.", UserWarning)
/usr/local/lib/python3.10/dist-packages/autogen/agentchat/conversable_agent.py:2553: UserWarning: Function 'solve_arithmetic_sequence' is being overridden.
  warnings.warn(f"Function '{func_sig['name']}' is being overridden.", UserWarning)
/usr/local/lib/python3.10/dist-packages/autogen/agentchat/conversable_agent.py:2514: UserWarning: Function 'evaluate_arithmetic_terms' is being overridden.
  warnings.warn(f"Function '{name}' is being overridden.", UserWarning)
/usr/local/lib/python3.10/dist-packages/autogen/agentchat/conversable_agent.py:2514: UserWarning: Function 'solve_arithmetic_sequence' is being overridden.
  warnings.warn(f"Function '{name}' is being overridden.", UserWarning)


assistant (to mathproxyagent):

To find \( a + b \) for the given piecewise function \( f(x) \), we need to ensure that the function is continuous at the points where the pieces change, specifically at \( x = 2 \) and \( x = -2 \).

1. **Check continuity at \( x = 2 \)**:
   - For \( x = 2 \), the second piece is defined as \( f(x) = x - 5 \).
   - Evaluate this piece at \( x = 2 \):
     \[
     f(2) = 2 - 5 = -3.
     \]

   - The first piece for \( x > 2 \) is \( f(x) = ax + 3 \). For continuity, we need:
     \[
     \lim_{x \to 2^+} f(x) = a(2) + 3.
     \]
     Setting this equal to the value from the second piece gives us:
     \[
     2a + 3 = -3.
     \]
     Solve for \( a \):
     \[
     2a = -3 - 3 = -6 \implies a = -3.
     \]

2. **Check continuity at \( x = -2 \)**:
   - The value from the second piece, valid from \( -2 \leq x \leq 2 \), is:
     \[
     f(-2) = -2 - 5 = -7.
     \]

   - The third piece for \( x < -2 \) is \( f(x) = 2x - b \). For continuity, we need:


In [24]:
sum = 0
for index, query in enumerate(test_data):
    is_correct = user_proxy.initiate_chat(recipient=assistant, answer=query["answer"], problem=query["question"])
    sum += is_correct
success_rate_with_agent_training = sum / 10

mathproxyagent (to assistant):

Let's solve a math problem.
Query requirements:
You should always use the 'print' function for the output and use fractions/radical forms instead of decimals.
You can use packages like sympy to help you.
You must follow the formats below to write your code:
```python
# your code
```
If some packages are missing, you could also suggest a code to install the corresponding package.

Please follow this process:
1. Solve the problem step by step (do not over-divide the steps).
2. Take out any queries that can be asked through Python code (for example, any calculations or equations that can be calculated) and functions you know in the context of this conversation.

Please
(1) do not mix suggested Python codes and function calls in one step.
(2) You MUST remember that you don’t have a function named "python" available.

You must follow the formats below to write your Python code:
```python
# your code
```

3. Wait for me to give the results or wait for the exec

In [25]:
print(
    "------------------------------------------------Functions learned------------------------------------------------"
)
for func in assistant.llm_config["functions"]:
    print(func["name"] + ": " + func["description"] + "\n")
print("------------------------------------------------Summary------------------------------------------------\n")
print("success_rate_without_agent_training: {average}%\n".format(average=success_rate_without_agent_training * 100))
print("success_rate_with_agent_training: {average}%\n".format(average=success_rate_with_agent_training * 100))

------------------------------------------------Functions learned------------------------------------------------
simplify_large_fraction: Simplifies a given large fraction or rational number.

calculate_and_round_investment: Calculate the required investment for a future value and return it as a rounded dollar amount.

calculate_and_return_investment: Calculate the investment needed for a desired future value and return it as a rounded dollar amount, with the option for output format.

calculate_investment: Calculate the principal investment needed for a desired future value with compound interest, given the future value, annual interest rate, compounding frequency, and time period.

compute_compound_growth: Computes the growth factor based on the annual interest rate, number of compounding periods, and time in years.

calculate_investment_multiples: Calculate the investment needed for a desired future value with compound interest, given the future value, annual interest rate, compoun